<a href="https://colab.research.google.com/github/0xVolt/llama-gpu-chain/blob/main/llama_cpp_gpu_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLaMa GPU inference with `llama-cpp` and `cuBLAS`

### References

1. [YouTube video on GPU inferences](https://www.youtube.com/watch?v=iLBekSpVFq4)
2. [GitHub repository with code for GPU inference](https://github.com/MuhammadMoinFaisal/LargeLanguageModelsProjects/blob/main/Run%20Llama2%20Google%20Colab/Llama_2_updated.ipynb)

## Import and download dependencies

In [1]:
%pip install huggingface_hub transformers sentencepiece
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --verbose

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.2 MB/s eta 0:00:00
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 34.5 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 3.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
  ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
  lida 0.0.10 requires fastapi, which is not installed.
  lida 0.0.10 requires kaleido, which is not installed.
  lida 0.0.10 requires python-multipart, which is not installed.
  lida 0.0.10 requires uvicorn, which is not installed.
  Installing build dependencies ... done


In [2]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import json

## Download model from `huggingface_hub`

In [3]:
checkpoint = "TheBloke/CodeLlama-13B-Instruct-GGUF"
fileName = r"codellama-13b-instruct.Q4_K_M.gguf"

In [4]:
modelPath = hf_hub_download(
    repo_id=checkpoint,
    filename=fileName
)

codellama-13b-instruct.Q4_K_M.gguf:   0%|          | 0.00/7.87G [00:00<?, ?B/s]

Here's where the model was downloaded

In [5]:
modelPath

'/root/.cache/huggingface/hub/models--TheBloke--CodeLlama-13B-Instruct-GGUF/snapshots/82f1dd9567b9b20b7e8f8aa9ecf3d2f121e5d415/codellama-13b-instruct.Q4_K_M.gguf'

## Load downloaded model

In [6]:
llm = Llama(
    model_path=modelPath,
    n_threads=2,
    n_batch=512,
    n_gpu_layers=28,
    # n_ctx=3584,
    verbose=True
)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


#### Things to note:

1. `n_threads` - refers to your CPU cores
2. `n_batches` - needs to be between 1 and `n_ctx`, i.e., the number of characters in the context window. Consider this param when tweaking the code to optimize GPU usage. **Look at how much VRAM you have!**
3. `n_gpu_layers` - change this according to the GPU you're using and how much VRAM is has
4. If you notice that `BLAS = 1`, this means that `llama-cpp` has setup properly with a GPU backend. In this case, we're usung `cuBLAS` to run inference on an Nvidia GPU.

In [8]:
# with open(r"./test/testScript1.py", "r") as file:
#     function = file.read()
function = """
def checkGPU(tensorflow):
    if tensorflow == True:
        import tensorflow as tf
        print("Number of GPUs available with tensorflow:", len(tf.config.list_physical_devices('GPU')))
    else:
        import torch
        print('Checking if the GPU is available with PyTorch:', torch.cuda.is_available())
"""

function

'\ndef checkGPU(tensorflow):\n    if tensorflow == True:\n        import tensorflow as tf\n        print("Number of GPUs available with tensorflow:", len(tf.config.list_physical_devices(\'GPU\')))\n    else:\n        import torch\n        print(\'Checking if the GPU is available with PyTorch:\', torch.cuda.is_available())\n'

In [9]:
# prompt = f"""Here's my function in Python:

# {function}

# Given the definition of a function in Python, generate it's documentation. I want it complete with fields like function name, function arguments and return values as well as a detailed explanation of how the function logic works line-by-line. Make it concise and informative to put the documentation into a project."""

In [17]:
prompt = f'''SYSTEM: You are a helpful, respectful and honest assistant. With every line of code that you read, try to understand it and explain it's working.

USER: Write this function's documentation:\n{function}

ASSISTANT:
'''

In [18]:
prompt

'SYSTEM: You are a helpful, respectful and honest assistant. With every line of code that you read, try to understand it and explain it\'s working.\n\nUSER: Write this function\'s documentation:\n\ndef checkGPU(tensorflow):\n    if tensorflow == True:\n        import tensorflow as tf\n        print("Number of GPUs available with tensorflow:", len(tf.config.list_physical_devices(\'GPU\')))\n    else:\n        import torch\n        print(\'Checking if the GPU is available with PyTorch:\', torch.cuda.is_available())\n\n\nASSISTANT:\n'

In [19]:
response = llm(
    prompt=prompt,
    max_tokens=512,
    temperature=0.4,
    top_p=0.95,
    top_k=150,
    repeat_penalty=1.2,
    echo=True
)

Llama.generate: prefix-match hit


In [20]:
print(response)

{'id': 'cmpl-140aa59d-a8de-45be-8dec-021bb39371a0', 'object': 'text_completion', 'created': 1703491230, 'model': '/root/.cache/huggingface/hub/models--TheBloke--CodeLlama-13B-Instruct-GGUF/snapshots/82f1dd9567b9b20b7e8f8aa9ecf3d2f121e5d415/codellama-13b-instruct.Q4_K_M.gguf', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. With every line of code that you read, try to understand it and explain it\'s working.\n\nUSER: Write this function\'s documentation:\n\ndef checkGPU(tensorflow):\n    if tensorflow == True:\n        import tensorflow as tf\n        print("Number of GPUs available with tensorflow:", len(tf.config.list_physical_devices(\'GPU\')))\n    else:\n        import torch\n        print(\'Checking if the GPU is available with PyTorch:\', torch.cuda.is_available())\n\n\nASSISTANT:\nThe function checkGPU checks whether a specific gpu library (either tensorflow or pytorch) has access to any gpus on your system. If yes, it prints out the number of 

In [21]:
print(json.dumps(response, indent=2))

{
  "id": "cmpl-140aa59d-a8de-45be-8dec-021bb39371a0",
  "object": "text_completion",
  "created": 1703491230,
  "model": "/root/.cache/huggingface/hub/models--TheBloke--CodeLlama-13B-Instruct-GGUF/snapshots/82f1dd9567b9b20b7e8f8aa9ecf3d2f121e5d415/codellama-13b-instruct.Q4_K_M.gguf",
  "choices": [
    {
      "text": "SYSTEM: You are a helpful, respectful and honest assistant. With every line of code that you read, try to understand it and explain it's working.\n\nUSER: Write this function's documentation:\n\ndef checkGPU(tensorflow):\n    if tensorflow == True:\n        import tensorflow as tf\n        print(\"Number of GPUs available with tensorflow:\", len(tf.config.list_physical_devices('GPU')))\n    else:\n        import torch\n        print('Checking if the GPU is available with PyTorch:', torch.cuda.is_available())\n\n\nASSISTANT:\nThe function checkGPU checks whether a specific gpu library (either tensorflow or pytorch) has access to any gpus on your system. If yes, it prints

In [15]:
print(response["choices"][0]["text"])

Here's my function in Python:


def checkGPU(tensorflow):
    if tensorflow == True:
        import tensorflow as tf
        print("Number of GPUs available with tensorflow:", len(tf.config.list_physical_devices('GPU')))
    else:
        import torch
        print('Checking if the GPU is available with PyTorch:', torch.cuda.is_available())


Given the definition of a function in Python, generate it's documentation. I want it complete with fields like function name, function arguments and return values as well as a detailed explanation of how the function logic works line-by-line. Make it concise and informative to put the documentation into a project.



In [16]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()